In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns

In [ ]:
df = pd.read_csv('Alzheimer.csv')
df

In [ ]:
df.info()

# Preprocessing Data

Label Encoder

In [ ]:
#LabelEncoder to Encode Multiple Columns
from sklearn.preprocessing import LabelEncoder

cols = ['Education', 'family history', 'Occupation', 'Triglycerides', 'smoking_status']

df[cols] = df[cols].apply(LabelEncoder().fit_transform)
df.head()

In [ ]:
#Check for missing values
df.isna().sum()

# Hanlde missing value of BMI Column

In [ ]:
df_filter = df.loc[df['BMI'].notnull(),:]
df_filter['alzheimer'].value_counts()

In [ ]:
#fill na in BMI column with mean
df = df.dropna(subset=['BMI'])

In [ ]:
df.isna().sum()

In [ ]:
df.alzheimer.value_counts()

In [ ]:
target_count = df.alzheimer.value_counts()
target_count.plot(kind='bar', title='Count (target)');

In [ ]:
sns.pairplot(data=df,
             diag_kind="kde")

# Create feature and label data

In [ ]:
#Remove HN of ID patiant and Label class
X = df.loc[:,~((df.columns=='alzheimer') | (df.columns=='HN'))]
X

In [ ]:
y = df['alzheimer']
y

In [ ]:
y.value_counts()

# Split Train-Test Data

In [ ]:
# Your code (use random_state=1234)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=1234)

In [ ]:
X_train.shape

In [ ]:
y_train.value_counts()

In [ ]:
X_test.shape

In [ ]:
y_test.value_counts()

In [ ]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Feature Data Scaling

In [ ]:
df.std(numeric_only=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
X_train_sc = scaler.transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc, columns=X_train.columns)

In [ ]:
X_test_sc = scaler.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc, columns=X_test.columns)

In [ ]:
X_train_sc.std(numeric_only=True)

In [ ]:
X_test_sc.std(numeric_only=True)

# Classification with ANN model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as metrics

In [ ]:
ann = MLPClassifier(activation='relu',
                    solver='adam',
                    random_state=1234, # random number generation for weights and bias initialization
                    learning_rate='adaptive',
                    max_iter=700
                   )

In [ ]:
skf = StratifiedKFold(n_splits=10,
                      shuffle=True,
                      random_state=1234)

In [ ]:
parameters = {'batch_size': [16, 32, 64, 128],
              'hidden_layer_sizes': [(24, 12, 6), (64, 24, 12, 6), (200, 50, 50, 25)]
              }

In [ ]:
grid_search = GridSearchCV(estimator = ann,
                           param_grid = parameters,
                           cv = skf)

In [ ]:
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Train model with best batch_size

In [ ]:
ann_optimal = MLPClassifier(hidden_layer_sizes=(24, 12, 6),
                    activation='relu',
                    solver='adam',
                    random_state=1234, # random number generation for weights and bias initialization
                    learning_rate='adaptive',
                    max_iter=700,
                    batch_size=16
                   )

In [ ]:
ann_optimal.fit(X=X_train_sc, y=y_train)

In [ ]:
y_pred_test = ann_optimal.predict(X_test_sc)
y_pred_test

In [ ]:
result_df = pd.DataFrame(y_pred_test)
result_df.value_counts()

# Performance of ANN model

In [ ]:
conf_mat = metrics.confusion_matrix(y_test, y_pred_test, labels=[0,1])

In [ ]:
metrics.ConfusionMatrixDisplay(conf_mat,
                               display_labels=[0,1]).plot()

In [ ]:
print(metrics.classification_report(y_test, y_pred_test))